In [1]:
!pip install -e /Users/lukehinds/dev/fabricsaas/deepfabric
!pip install datasets

Obtaining file:///Users/lukehinds/dev/fabricsaas/deepfabric
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for deepfabric (pyproject.toml) ... done
  Created wheel for deepfabric: filename=deepfabric-3.13.0-py3-none-any.whl size=9913 sha256=a502cac6989d6a501d28210d203e0494f7561fb708a788c2663d25c34314f465
  Stored in directory: /private/var/folders/tz/9sqh29bj7fg55k8vzr0mzpmw0000gn/T/pip-ephem-wheel-cache-pwrxuio1/wheels/54/67/61/de0d6f3db886f5a9af167f2cde5237914ad5a6b1d611d1c175
Successfully built deepfabric
  Attempting uninstall: deepfabric
    Found existing installation: deepfabric 3.13.0
    Uninstalling deepfabric-3.13.0:
      Successfully uninstalled deepfabric-3.13.0

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip in

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

from deepfabric.evaluation import Evaluator, EvaluatorConfig, InferenceConfig

ds = load_dataset("lukehinds/test-generic-tools", split="train")
splits = ds.train_test_split(test_size=0.1, seed=42) # type: ignore
train = splits["train"]
eval_ds = splits["test"]

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
def format_example(example):
    # Clean messages - remove None values
    clean_messages = []
    for msg in example["messages"]:
        clean_msg = {k: v for k, v in msg.items() if v is not None}
        clean_messages.append(clean_msg)

    return {"text": tokenizer.apply_chat_template(
        clean_messages, tokenize=False, add_generation_prompt=False
    )}

formatted = train.map(format_example)
print(f"Sample formatted example:\n{formatted[0]['text']}")

# evaluate

# Configure evaluator (no dataset_path needed)
config = EvaluatorConfig(
    model_path="your-fine-tuned-model",
    inference_config=InferenceConfig(backend="vllm"),
)

# Run evaluation with HF Dataset directly
evaluator = Evaluator(config)
results = evaluator.evaluate(dataset=eval_ds)

# Access results
print(f"Tool Selection Accuracy: {results.metrics.tool_selection_accuracy:.2%}")
print(f"Parameter Accuracy: {results.metrics.parameter_accuracy:.2%}")

Sample formatted example:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the current temperature in London and what's 25% of 200?<|im_end|>
<|im_start|>assistant
<tool_call>
{"name": "get_weather", "arguments": "{\"location\": \"London\", \"time\": \"current\"}"}
</tool_call>
<tool_call>
{"name": "calculate", "arguments": "{\"expression\": \"25% of 200\"}"}
</tool_call><|im_end|>
<|im_start|>user
<tool_response>
The current weather in London is partly cloudy with a temperature of 15°C and a wind speed of 10 km/h.
</tool_response>
<tool_response>
50
</tool_response><|im_end|>
<|im_start|>assistant
The current temperature in London is 15°C. 25% of 200 is 50.<|im_end|>



In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load from Hub
dataset = load_dataset("lukehinds/test-generic-tools", split="train")

# Split into train/eval
splits = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = splits["train"]
eval_ds = splits["test"]

# Format using your tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

def format_example(example):
    messages = [{k: v for k, v in msg.items() if v is not None}
                for msg in example["messages"]]
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

formatted_train = train_ds.map(format_example)
print(f"Sample formatted example:\n{formatted_train[0]['text']}")

Sample formatted example:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the current temperature in London and what's 25% of 200?<|im_end|>
<|im_start|>assistant
<tool_call>
{"name": "get_weather", "arguments": "{\"location\": \"London\", \"time\": \"current\"}"}
</tool_call>
<tool_call>
{"name": "calculate", "arguments": "{\"expression\": \"25% of 200\"}"}
</tool_call><|im_end|>
<|im_start|>user
<tool_response>
The current weather in London is partly cloudy with a temperature of 15°C and a wind speed of 10 km/h.
</tool_response>
<tool_response>
50
</tool_response><|im_end|>
<|im_start|>assistant
The current temperature in London is 15°C. 25% of 200 is 50.<|im_end|>

